# Data Cleaning with Pandas Tutorial

To edit this notebook:
- Save the notebook by selecting `Download .ipynb` from the `File` tab
- Go to [Colaboratory](https://colab.research.google.com/) and upload the notebook from the `File` tab
- Alternatively, you can import the notebook to your Google Drive and select `Open with` when you right-click. Select `Colaboratory` or `+ Connect more apps` to install Colaboratory first

## Install the Database Module

The code below installs a postgres database module to allow our notebook to connect to the Strata Scratch database



In [1]:
!pip install psycopg2

## Import Required Modules

Import a few required modules that enables us to query data and perform analytics

In [2]:
import numpy as np
import pandas as pd
import psycopg2 as ps

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Connect to Strata Scratch

Make sure to enter your username and database password. Your database password is not the same as your login password. You can find your database password in the Profile tab once logged into Strata Scratch. 

In [3]:
host_name = 'db-strata.stratascratch.com'
dbname = 'db_strata'
port = '5432'
user_name = '' #enter username
pwd = '' #enter your database password found in the profile tab in Strata Scratch

try:
    conn = ps.connect(host=host_name,database=dbname,user=user_name,password=pwd,port=port)
except ps.OperationalError as e:
    raise e
else:
    print('Connected!')

OperationalError: ignored

## Pull Data From Strata Scratch

#### Enter SQL code below to pull the dataset you're interested in

If you get an error, it likely means that the connection timed out. Try connecting to Strata Scratch again before executing the code below.

A list of datasets is found in SQL LAB in Strata Scratch.

In [0]:
#Write SQL below to pull datasets 
cur = conn.cursor()
cur.execute(""" 
            SELECT *  FROM nfl_combine; 
            """)
data = cur.fetchall()
colnames = [desc[0] for desc in cur.description] 
conn.commit()

#create the pandas dataframe
data = pd.DataFrame(data)
data.columns = colnames

#close the connection
cur.close()

## Check To See If Your Pulled The Dataset

Your dataset should be in a pandas dataframe named `data`

In [0]:
data.head()

# Data Cleaning Using Pandas

## Understanding your data types

To see what data types our columns have we can use the `pd.DataFrame.dtypes` property.

In [0]:
data.dtypes

We see 3 different types which are present in this dataset (and which are most common everywhere)
- int64 which is a integer number with a sign (can be positive or negative) - In SQL corresponds to INTEGER
- float64 which is a real number - In SQL corresponds to NUMERIC
- object which can be anything but is usually a string - In SQL corresponds to TEXT

One **very important** thing to know is how pandas represents columns with missing values. 
- If all present values in a column are numbers (integer or real doesn't matter) then it treats that column as `float64`
- If at least one is non-numeric it treats the column as type `object`.

When downloading from database like we do in this course pandas will **almost** always honor the types in the database. Notice the bolded almost because there are times when we must manually change the types for various reasons. 

## Type conversions in Pandas

We will use the method `astype`.

Syntax is: `data[column_name] = data[column_name].astype(new_type)`

Types in pandas come from numpy so `new_type` looks like:
- np.int64
- np.float64
- np.object

You can convert only between compatible types if there are **no missing values** and the values are convertible.

For example: you can conver 42 to a real number and you can convert "123.45" to a real number but you can't convert "hello" to any number.

Generally everything can be converted to object though you will rarely need to do that and usually you go the other way around.

In [0]:
data["heightinches"].astype(np.int32).head()

In [0]:
data["heightinches"].astype(np.object).head()

## Cleaning strings

Strings are very often typed into your computer by people and the data you get is full of typos and weird formats. You will learn about regular expressions which are an effective tool to fix many of these problems and you will see how to convert strings to numbers and dates.

### Regular expressions

Regular expressions or regexp as they are often called are special strings which can either filter lists of strings or transform strings with a bit of help from other code.

In python all functionality related to regexp lives in the `re` module which we will now import.

- https://docs.python.org/3.7/library/re.html

In [0]:
import re

Before we can use regexp we must learn of their special structure.

Let's start with the simplest possible example.

In [0]:
re.match(pattern="abc", string="abcdef")

`re.match` finds matches of our regular expression (`pattern` paramater) and a string of interest (`string` parameter)

The results are two values:
- span which tells us where the match starts and where it ends
- match which is what was matched

Here we used the simplest regexp, a sequence of characters. 

#### Regex Choice

The power of regex comes from choice, repetition and capture groups.

To combine two sequences into a choice we use the pipe (|).

The following code will match both Trick and Treat.

Notice that we had to put both in parentheses in the regexp pattern because if we didn't we would implicitly get:
- Tric(k|T)reat 

We also use a new method called `re.findall`. It has the same parameters as match except that it finds all matches while `match` finds only the first one.

The brackets in the output will be explained along capture groups.

In [0]:
re.findall(pattern="(Trick)|(Treat)", string="Trick or Treat")

#### Regex repetition

For example take the string:
- aaaabbbb

It is 4 'a' followed by 4 'b'

We can match it by writing the whole string as the pattern but let's generalize.

We want to match _n_ times a and _n_ times b.

The pattern for this case is:
- a{4}b{4}
- Generally a{n}b{n}

In [0]:
re.match(pattern="a{4}b{4}", string="aaaabbbb")

#### Unbounded repetition

Sometimes we want to have an unlimited amount of some character (or a sequence of characters).

To do that we use the `*` operation.

`a* = a|(aa)|(aaa)|(aaaa)|...`

There is also the `+` operation which is unbounded but requires at least one occurence while `*` requires 0 occurences.

We can rewrite our example from above to use `*` and notice how we now have a higher number of a but it still matches.

In [0]:
re.match(pattern="a*b{4}", string="aaaaaaaaaaabbbb")

#### Capture groups

When you surround some characters with brackets you define a capture group which you can later get via `match_result.group(idx)`.

### Parsing with RegExp

This was a lot of theory but luckily we now get to real examples where this stuff should get clearer.

#### Special symbols

You can use so called special symbols to save some typing. There are a lot of them but we will only show the most important ones here:
- \d is any number and is equivalent to 0|1|2|3|4|5|6|7|8|9
- \w is any letter
- \s is any space
- .  is any character except new line

#### Example 1

Extract the components of a telephone number.

Telephone numbers are in format XXX-XXX-XXX where X is some number.

The pattern we use is (\d{3})-(\d{3})-(\d{3}) which means number 3 times followed by a dash folowed by number 3 times followed by dash and followed by number 3 times.

Capture groups are here to get parts of the number.

In [0]:
re.findall(pattern="(\d{3})-(\d{3})-(\d{3})", string="123-456-789")

In [0]:
parts = re.findall(pattern="(\d{3})-(\d{3})-(\d{3})", string="123-456-789")[0]

parts[0]

#### Example 2

Suppose we are constrained that the first 3 numbers must be one of 444, 666, 999.
- (9{3})|(6{3})|(4{3}) is the addition to satisfy this requirement

Notice that we now have more capture groups so you can see empty values in the output.

In [0]:
re.findall(pattern="((9{3})|(6{3})|(4{3}))-(\d{3})-(\d{3})", string="123-456-789")

In [0]:
re.findall(pattern="((9{3})|(6{3})|(4{3}))-(\d{3})-(\d{3})", string="444-456-789")

#### Example 3

Find all players whose colleges have names equal to any 3 uppercase letters. For sake of example let's assume all other colleges are noise in our data.

When you hear the word any the dot (.) must always come to mind but when you read a bit more you see they want only uppercase letters. 

So you start to write (A|B|C...){3} but stop because there is a quicker way to do this.
[A-Z] is equal to (A|B|C|...|Z) and it also works for numbers [0-9] = 0|1|2|...|9

So the pattern is ([A-Z]{3}).

We use the `apply` function which is explained later but the idea is to build a new boolean numpy array with which to index the dataframe.

The parts important for this section are:
- re.fullmatch which passess as a match only if the whole string matches, not parts of it.
- is not None test. When there is no match `re.match` and `re.fullmatch` return None.

In [0]:
boolean_index_3letter_colleges = data["college"].apply(lambda c: 
                                                       re.fullmatch(pattern="[A-Z]{3}", 
                                                                    string=str(c))
                                                       is not None)

data[boolean_index_3letter_colleges]

#### Example 4

Find all people who have initials as their first name.

The wanted names look like:
- B.J.
- J.J.
- J.R.
- etc

You can match for dots using regex but because the dot is a special character you must "escape" it. Escaping means putting \ before the dot (that is \. is the escaped dot).

The pattern is a sequence of:
- [A-Z] (any uppercase letter)
- \. (escaped dot) [unescaped dot means match any character]
- [A-Z] (again any uppercase letter)
- \. another dot

In [0]:
boolean_index_initials = data["firstname"].apply(lambda c: 
                                                       re.fullmatch(pattern="[A-Z]\.[A-Z]\.", 
                                                                    string=str(c))
                                                       is not None)

data[boolean_index_initials]

### Strings to numbers

You can convert a string to integer using the `int` function.

In [0]:
int("123")

You can convert a string to integer using the `float` function.

In [0]:
float("123")

Performing casts using `astype` can also transform strings to numbers.

In [0]:
np.array(["123", "123", "123"]).astype(np.float64)

#### Extracting numbers from strings - Measurements

Lets say you have a string like "13ft" or "pills=15" or even more complex like "15 pills, 15 injections".

The easiest way to extract these numbers is using regular expressions.

Let's start with the simplest one "13ft". The pattern is any number repeated as much as needed. This is easy because we can use the `span` function.

In [0]:
measurement = "13ft"

span = re.match(pattern="\d*", string=measurement).span()

print(span)

measurement[span[0]:span[1]]

pills=15

Here we use findall which returned us with a single number the number of pills.

In [0]:
measurement = "pills = 15"

match = re.findall(pattern="pills = (\d+)", string=measurement)[0]

match

"15 pills, 15 injections"

Again we will use findall but notice our pattern: \d`*` and that's it.

Regex can be as easy and as complex as you make so give yourself time to think about what happens when you write your own regexes.

In [0]:
measurement = "15 pills, 15 injections"

match = re.findall(pattern="\d+", string=measurement)

match

Our regex above has a problem and that is we don't know what number corresponds to what measurement type.

We can fix it by extending our regex and using capture groups.

So what's [^,] all about?

Remember when we said [A-Z] are all uppercase letters. When you put ^ after the opening bracket the meaning changes to everything but the letters in brackets (e.g. [^A-Z] is everything except upper case letters, [^,] everything except a comma)

In [0]:
measurement = "15 pills, 15 injections"

matches = re.findall(pattern="(\d+) ([^,]+)", string=measurement)

matches

Generally using findall is better than match for this type of tasks.

## Filll missing values

#### Fill missing values for college with 'No College'

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html

`fillna` method when given a single value like in this example fills all missing values in the dataframe with that value. 

When given a dictionary to the value parameter it replaces according to that dictionary.

`inplace=True` holds the same implications as in `sort_values`. 

In [0]:
data.fillna(value='No College', inplace=True)

## Dropping rows on missing values

### Remove players that have null values for the pick

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html

Dropna will drop rows according to some null value criteria.

That criteria is defined via the `subset` parameter which is a list of columns.
- `how="any"` means delete row if at least one value is null.
- `how="all"` means delete row if all values are null.

`inplace=True` holds the same implications as in `sort_values`.

In [0]:
data.dropna(how='any', subset=['pick'], inplace=True)

## Replacing values in string columns

### Replace RB and QB with Running Back and Quarterback

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html

This method will change **all values** in the dataframe using a set of rules of the form:
- Replace to_replace[0] with value[0]
- Replace to_replace[1] with value[1]
- ...

`inplace=True` holds the same implications as in `sort_values`. 

In [0]:
data.position.replace(to_replace=['RB','QB'],
                      value=['Running Back','Quarterback'], 
                      inplace=True)
data.head()

## Using dummy variables

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html

Dummy variables are useful to convert a single multi-value column to multiple binary valued columns.

In normal person speak that would mean assume you had the following data (genders):
- M
- F
- M
- M
- U
- F

There are three unique values. Binarizing with prefix='Gender' will make 3 new columns:
- Gender_M
- Gender_F
- Gender_U

Each row in the data can have only a single value of 1 in these 3 columns with others being 0s.

Our data from above would look like the following table when binarized:

|Original value|Gender_M|Gender_F|Gender_U|
|-|--------|--------|--------|
|M|1|0|0|
|F|0|1|0|
|M|1|0|0|
|M|1|0|0|
|U|0|0|1|
|F|0|1|0|

### Create dummy values for position

Observe in this example as we make dummy variables for `position` with prefix='Pos'

In [0]:
dummy_data = pd.get_dummies(data.position, prefix='Pos')
dummy_data.head()

#### Example use case for dummies

Calculate the total number of players per position.

If you put `prefix=None` the values from positions columns are now column names with no prefix.

In [0]:
pd.get_dummies(data.position, prefix=None).sum()

### Merge the dummy data with the original data set

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html

Merge is pandas speak for sql join.

The main parameters for merge are:
- dataframe_left which is `data` in our case
- dataframe_right which is `data` in our case
- how which describes the type of SQL JOIN to perform. Some possible values are "left", "right", "outer", "inner"

With that in place only thing left to decide is what to JOIN on.
- We must choose the join keys for both dataframes.
- One choice is to use the index as we do here (for both dataframes actually(
- Another choice is to use columns which is controlled via the parameters left_on and right_on.
- You must not set both left_on and left_index. Same for right.

In a simplified case when you have columns with same name in both dataframes you can use only the `on` parameter.

In [0]:
data.merge(dummy_data, how='inner', left_index=True, right_index=True) 

## Converting values

### Convert weight from lbs to kg

Remmeber: pandas columns are series but they can do all arithmetic stuff that numpy arrays can do.

In [0]:
data['weight_kg'] = data.weight/2.2
data.head()

### Capitalize name

- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html

Apply is a very versatile function which allows arbitrary data transformations.

The main parameters of apply are the function and the axis.

When dealing with columns as in this example, there is no axis, but when using apply on the whole dataframe we must use either axis=1 which means apply over rows or axis=0 which means apply over columns.

In this example we apply `str.upper` over the values in the name column.

In [0]:
# need the apply function if you're doing an operation on a column. Python doens't know to apply it to the entire dataset.

data.name.apply(str.upper).head()

### Lambda Functions

Lambda functions are short, compact functions in python. They can have any number of arguments but only one expression. The expression is evaluated and returned. Lambda functions can be used wherever function objects are required.

Example:
```
double = lambda x: x * 2

print(double(5))
```



Output would be `10`

This lambda function will double any value in `x`. 

#### Reverse order of first and last name

`lambda x:'{0}, {1}'.format(x['lastname'], x['firstname'])` 
- x is now a row (**axis=1**) which is of type Series. we can access firstname as x['firstname'] or x.firstname as in any other series.
- The format method in python replaces {i} with i-th argument. So {0} gets replaced with first argument which is x['lastname'] and {1} will get replaced with x['firstname']
- Thus this function takes a row as input and outputs a string which contains the last name followed by the first name

We apply this function over all rows (**axis=1**) and obtain a series as a result.

We store that series in our dataframe with the assignment.

In [0]:
data['last_firstname'] = data.apply(lambda x:'{0}, {1}'.format(x['lastname'], x['firstname']), axis=1)
data.head()